In [ ]:

import functools
import os
import warnings

import geopandas as gpd
import ipywidgets as widgets
import pandas as pd
import matplotlib
from matplotlib import animation
from matplotlib.lines import Line2D
from matplotlib.pyplot import ylabel
from matplotlib.text import Text
import matplotlib.pyplot as plt
import numpy as np

from iwfm_lwa import (skip_until_flag, get_groundwater_nodes, load_gwalloutfl, get_stratigraphy, get_stream_nodes,
                      get_var, line_to_list)

warnings.filterwarnings('ignore')


# Path to the root of the repository
repo_dir = os.getcwd()
current_dir = os.getcwd()



while os.path.split(os.path.split(repo_dir)[0])[1] == "c2vsim_fg_stream_transects":
    repo_dir = os.path.split(repo_dir)[0]


os.chdir(repo_dir)

from functions.add_bore_logs import add_bore_logs
from functions.make_transect_layout import make_transect_layout

os.chdir(current_dir)

# Let's set key paths
data_dir = os.path.join(repo_dir, "data")


# Path to Butte Creek stream nodes file
butte_creek_stream_nodes_csv_path = os.path.join(data_dir, "butte_creek_stream_nodes.csv")

# Path to Butte Creek simulated groundwater heads
butte_creek_wt_stream_nodes_csv_path = os.path.join(data_dir, "butte_creek_gwallout_stream_nodes.csv")

# Path to Butte Creek CASGEM observations lookup table
obs_wells_butte_creek_shp_path = os.path.join(data_dir, "obs_wells_butte_creek.shp")

# Path to Butte Creek CASGEM observed groundwater levels
obs_wells_butte_creek_csv_path = os.path.join(data_dir, "obs_wells_butte_creek.csv")

# Path to Butte Creek lithology logs shapefile
lith_logs_butte_creek_shp_path = os.path.join(data_dir, "lith_logs_butte_creek.shp")

# Path to Butte Creek AEM lithology logs
butte_lith_csv_path = os.path.join(data_dir, "butte_creek_lithology.csv")

# Butte Creek Stream Transect
In this section, we will make a stream transect for Butte Creek using the C2VSim-FG model streambed and groundwater
table elevations, DWR CASGEM groundwater levels observations, and DWR's AEM lithology logs.

In [ ]:

# Let's load the stream nodes for Butte Creek
stream_nodes_df = pd.read_csv(butte_creek_stream_nodes_csv_path)

# Let's load the simulated groundwater table at the stream nodes
sim_df = pd.read_csv(
    butte_creek_wt_stream_nodes_csv_path,
    parse_dates=["date"]
)

# Let's load the lithology logs
lith_df = pd.read_csv(butte_lith_csv_path)

# Let's load the lithology logs shapefile
lith_lut_df = gpd.read_file(lith_logs_butte_creek_shp_path)

obs_lut_df = gpd.read_file(obs_wells_butte_creek_shp_path)

obs_df = pd.read_csv(obs_wells_butte_creek_csv_path, parse_dates=["date"])

# Let's make the template for the transect

kwargs = {}

date_col = kwargs.pop("date_col", "date")
obs_col = kwargs.get("obs_col", "gwe")
obs_kwargs = kwargs.get("obs_kwargs", None)
proj_d_col = kwargs.get("proj_d_col", "proj")
sim_col = kwargs.get("sim_col", "wte_ft")
sim_kwargs = kwargs.get("sim_kwargs", None)
title = kwargs.get("title", "Butte Creek Stream Transect")
title_kwargs = kwargs.get("title_kwargs", None)
well_id_col = kwargs.get("well_id_col", "site_code")
node_key_col = kwargs.pop("node_key_col", "igw")
interval = kwargs.pop("interval", 500)  # milliseconds between frames

if obs_kwargs is None:
    obs_kwargs = {"color": "#4B7164"}
    kwargs["obs_kwargs"] = obs_kwargs

if sim_kwargs is None:
    sim_kwargs = {
        "color": "navy",
        "label": "Simulated GWE"
    }
    kwargs["sim_kwargs"] = sim_kwargs

if title_kwargs is None:
    title_kwargs = {"pad": 150}
    kwargs["title_kwargs"] = title_kwargs

fig, axes = make_transect_layout(
        stream_nodes_df,
        sim_df,
        obs_df,
        obs_lut_df,
        lith_lut_df,
        lith_df, **kwargs)

sim_kwargs.pop("label")

# Now, we make the update function that will make the frames in the animation
def update(
        ts,
        obs_df,
        obs_lut_df,
        sim_df,
        date_col=date_col,
        proj_d_col=proj_d_col,
        sim_col=sim_col,
        title=title,
        title_kwargs=title_kwargs,
        obs_kwargs=obs_kwargs,
        well_id_col=well_id_col):

    artists = []
    # List of artists labels that we need to remove every timestep
    artists_to_remove = obs_lut_df[well_id_col].tolist() + [sim_col]  + [title]
    # Let's remove artists from previous frame
    for artist in axes[0].get_children():
        artist_label = artist.get_label()
        if artist_label in artists_to_remove:
            try:
                artist.remove()
            except ValueError:
                pass

    # Let's add observations for the given timestep
    obs_df_ts = obs_df[obs_df[date_col] == ts].reset_index(drop=True)
    for well in obs_df_ts[well_id_col].unique().tolist():
        x = obs_lut_df.loc[obs_lut_df[well_id_col] == well, proj_d_col].values[0]
        obs = obs_df_ts.loc[obs_df_ts[well_id_col] == well, obs_col].values[0]
        artists.append(axes[0].scatter(x, obs, label=well, **obs_kwargs))

    # Let's add simulated groundwater levels for the given timestep
    sim_df_ts = sim_df[sim_df[date_col] == ts].reset_index(drop=True)
    # Let's join with stream nodes to get the projection distance
    sim_df_ts = pd.merge(
        sim_df_ts,
        stream_nodes_df[[proj_d_col, node_key_col]],
        on=node_key_col,
        how="left")
    artists.append(
        axes[0].plot(
            sim_df_ts[proj_d_col],
            sim_df_ts[sim_col],
            label=sim_col,
            **sim_kwargs
        )
    )


    #let's set the new title label for the given timestep
    title = f"{ts.strftime('%Y %B')}"
    artists_to_remove[len(artists_to_remove)-1] = title
    artists.append(axes[0].set_title(title , **title_kwargs))

    return artists

ani = animation.FuncAnimation(
    fig=fig,
    func=functools.partial(
        update,
        obs_df=obs_df,
        obs_lut_df=obs_lut_df,
        sim_df=sim_df
    ),
    frames=sim_df[date_col].unique(),
    interval=interval,
)


html_video = ani.to_html5_video()

plt.close()

widgets.HTML(
        value=html_video,
        placeholder="",
        description="",
    )